In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
import h5py
import numpy as np

from ffn.inference import storage
from ffn.inference.segmentation import clear_dust, make_labels_contiguous, clean_up

#from cc3d import connected_components


import neuroglancer


import cloudvolume

def hdf5_to_cloudvolume():
  pass

import matplotlib.pyplot as plt



# INPUT 
Probably the only part of the script you need to change anything

In [13]:
#RAW data information
raw_data = 'training_data.h5'
raw_data_dset = 'image' #usually don't change

##FFN segmentation information
seg_folder = './results/inference_01/'
seg_seed = (0,0,0)

#RESOLUTION information
xy_res = 1200
z_res = 1200

#PROCESSING information
cleanup = 0 #0 for nothing , 1 for SS-RAF, 2 for michal, 3 for michal+raf
min_particle = 10


# PROCESSING PART
Please don't touch!!!

In [14]:
#don't touch this lines
h5file = h5py.File(raw_data, 'r+')
image = h5file[raw_data_dset]
print('Raw data info:')
print(image.shape, image.dtype)


Raw data info:
(128, 128, 128) uint8


In [15]:
#don't touch this lines
seg, _ = storage.load_segmentation(seg_folder,seg_seed,allow_cpoint=True)
print('Label data info:')
print(seg.shape, seg.dtype)

Label data info:
(128, 128, 128) uint64


In [16]:

#don't change

seg_cleanup = np.zeros(seg.shape)
seg_cleanup = seg[:,:,:]


if(cleanup==0):
    pass

elif(cleanup==1):
    labels_in = np.zeros(seg.shape,dtype='uint8')
    labels_in[seg!=0] = 1
    labels_out = connected_components(labels_in)
    seg_cleanup = clear_dust(labels_out,min_particle)
    
elif(cleanup==2):    
    clean_up(seg_cleanup, min_size=min_particle)
    
elif(cleanup==3):
    clean_up(seg_cleanup, min_size=min_particle)
    
    labels_in = np.zeros(seg_cleanup.shape,dtype='uint8')
    labels_in[seg_cleanup!=0] = 1
    seg_cleanup = connected_components(labels_in)
    
else:
    print("put a valid cleanup option!!")

ids  = np.unique(seg, return_counts=1)
ids0 = np.unique(seg_cleanup,return_counts=1)

print ('Original dataset info:')
print(seg.shape,seg.dtype)
print ('Number of features:')
print (len(ids[0]))

print ('Cleanup dataset info:')
print(seg_cleanup.shape,seg_cleanup.dtype)
print ('Number of features:')
print (len(ids0[0]))


Original dataset info:
(128, 128, 128) uint64
Number of features:
67
Cleanup dataset info:
(128, 128, 128) uint64
Number of features:
67


# Visualization
Always execute viewer.close() after you leave otherwise you may leave hell-spawns on your memory.

In [17]:
voxel=(z_res,xy_res,xy_res)
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(
    source=neuroglancer.LocalVolume(data=image, voxel_size=voxel, volume_type='image'))
  s.layers['segmentation_clean'] = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(data=seg_cleanup, voxel_size=voxel, volume_type='segmentation'),segments=ids0[0])
  s.layers['segmentation_original'] = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(data=seg, voxel_size=voxel, volume_type='segmentation'),segments=ids[0])
print(viewer.get_viewer_url())

http://127.0.0.1:45733/v/ab9f1dfa633f2b4408897b657e39d36eed075c86/


ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/rlab/anaconda3/envs/klab/lib/python3.7/site-packages/tornado/iostream.py", line 713, in _handle_events
    self._handle_write()
  File "/home/rlab/anaconda3/envs/klab/lib/python3.7/site-packages/tornado/iostream.py", line 1063, in _handle_write
    self._write_buffer.advance(num_bytes)
  File "/home/rlab/anaconda3/envs/klab/lib/python3.7/site-packages/tornado/iostream.py", line 184, in advance
    assert 0 < size <= self._size
AssertionError
ERROR:asyncio:Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/home/rlab/anaconda3/envs/klab/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/home/rlab/anaconda3/envs/klab/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 122, in _handle_events
    handler_func(fileobj, events)
  File "/home/rlab/anaconda3/envs/klab

In [18]:
del viewer

# Tiff stack save!!

In [19]:
# dxchange.write_tiff_stack(image,'./stacks/results/cube_', dtype='uint8', overwrite=True)
# dxchange.write_tiff_stack(labels_out,'./stacks/results/seg_', dtype='uint64', overwrite=True)


# WELCOME TO DISNEYLAND
## Where dreams come true

In [20]:
from knossos_utils import knossosdataset
#magic happens here.. Magic happens everywhere.. 

knossos_location = './kcube/'
knossos_conf =  'knossos.conf'
knossos_new_anno = 'inference01'



#don't touch bellow
kd = knossosdataset.KnossosDataset()
kd.initialize_from_knossos_path(knossos_location+knossos_conf)

box_offset = [0,0,0] #don't touch, for future use

kd.from_matrix_to_cubes(offset=box_offset, data=seg, as_raw=False, kzip_path=os.path.join(knossos_location,knossos_new_anno))



/home/rlab/Documents/workspace/knossos_utils/knossos_utils/knossosdataset.py:1911: UserWarning: You are using implicit channel selection. This possibility will soon be removed. Please call set_channel() before reading or writing data using KnossosDataset.
  warnings.warn('You are using implicit channel selection. This possibility will soon be removed.'


box_offset: [0. 0. 0.]
box_size: [128 128 128]
start_cube: [0 0 0]
end_cube: [1 1 1]
box_offset: [0. 0. 0.]
box_size: [64 64 64]
start_cube: [0 0 0]
end_cube: [1 1 1]
box_offset: [0. 0. 0.]
box_size: [32 32 32]
start_cube: [0 0 0]
end_cube: [1 1 1]
